In [ ]:
# Train model
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
# from models import pretrained_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)


In [ ]:
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
from services.training_ensemble_model_service import train_and_evaluate_ensemble
from utils.preprocess import get_data_generators
from tensorflow.keras.utils import to_categorical

# Call get_data_generators to load the data
train_generator, validation_generator, label_encoder = get_data_generators(metadata_path=r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv', target_size=(224, 224), batch_size=32)

# Get one batch of data for training and validation
train_data, train_labels = next(train_generator)  # Correct usage of next()
val_data, val_labels = next(validation_generator)  # Correct usage of next()

# One-hot encode the labels
train_labels = to_categorical(train_labels, num_classes=7)
val_labels = to_categorical(val_labels, num_classes=7)

# Use validation data as the test set
test_data = val_data
test_labels = val_labels

# Train and evaluate the ensemble model
train_and_evaluate_ensemble(train_data, train_labels, val_data, val_labels, test_data, test_labels)
